# 线性回归 (Linear Regression)

---

## 📚 学习目标

通过本教程，你将学会：
- 理解线性回归的数学原理
- 掌握正规方程法求解线性回归
- 使用NumPy实现线性回归
- 使用Scikit-learn快速构建模型
- 评估和可视化回归结果

## 📋 前置知识

- Python基础语法
- NumPy基本操作
- 线性代数基础（矩阵乘法、矩阵求逆）

## ⏱️ 预计时间

25-35分钟

---

## 📖 第1部分：理论背景

### 什么是线性回归？

线性回归是一种**监督学习算法**，用于建立输入特征和连续输出目标之间的线性关系。它是最简单也是最常用的机器学习算法之一，是理解更复杂模型的基础。

### 数学模型

对于**单变量线性回归**，模型可以表示为：

$$h_\theta(x) = \theta_0 + \theta_1 x$$

其中：
- $h_\theta(x)$ 是预测值
- $\theta_0$ 是截距（偏置项）
- $\theta_1$ 是斜率（权重）
- $x$ 是输入特征

对于**多变量线性回归**，可以用向量形式表示：

$$h_\theta(\mathbf{x}) = \theta^T \mathbf{x} = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + ... + \theta_n x_n$$

### 损失函数

使用**均方误差(MSE)**作为损失函数：

$$J(\theta) = \frac{1}{2m}\sum_{i=1}^{m}(h_\theta(x^{(i)}) - y^{(i)})^2$$

其中 $m$ 是样本数量。

### 求解方法

#### 1. 正规方程法（解析解）

$$\theta = (X^TX)^{-1}X^Ty$$

**优点：**
- 一步到位，无需迭代
- 不需要选择学习率

**缺点：**
- 特征数量大时计算慢 $O(n^3)$
- 需要计算矩阵的逆

#### 2. 梯度下降法（迭代优化）

$$\theta_j := \theta_j - \alpha\frac{\partial}{\partial\theta_j}J(\theta)$$

**优点：**
- 特征数量大时也能高效计算
- 可以在线学习

**缺点：**
- 需要选择学习率
- 需要多次迭代

> 💡 **提示**: 对于线性回归问题，由于损失函数是凸函数，梯度下降法可以保证找到全局最优解。

---

## 💻 第2部分：代码实现

### 步骤1: 导入库和配置

In [ ]:
# ============================================================
# 导入必要的库
# ============================================================

# 数值计算
import numpy as np

# 可视化
import matplotlib.pyplot as plt

# 机器学习
from sklearn.linear_model import LinearRegression

# ============================================================
# 配置参数
# ============================================================

# 设置随机种子，确保结果可重复
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# 数据生成参数
N_SAMPLES = 100        # 样本数量
TRUE_INTERCEPT = 4.0   # 真实截距
TRUE_SLOPE = 3.0       # 真实斜率
NOISE_LEVEL = 0.5      # 噪声水平

# 可视化配置
FIGURE_SIZE = (10, 6)
plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文显示
plt.rcParams['axes.unicode_minus'] = False    # 负号显示

print("✓ 库导入和配置完成")

### 步骤2: 生成模拟数据

我们生成符合线性关系 $y = 4 + 3x + \epsilon$ 的数据，其中 $\epsilon$ 是高斯噪声。

- 真实截距 $\theta_0 = 4.0$
- 真实斜率 $\theta_1 = 3.0$

In [ ]:
# ============================================================
# 数据生成
# ============================================================

# 生成特征X：100个样本，范围[0, 2]
X = 2 * np.random.rand(N_SAMPLES, 1)

# 生成目标y：y = 4 + 3x + 高斯噪声
y = TRUE_INTERCEPT + TRUE_SLOPE * X + NOISE_LEVEL * np.random.randn(N_SAMPLES, 1)

# 数据检查
print("✓ 数据生成完成")
print(f"  样本数量: {N_SAMPLES}")
print(f"  X形状: {X.shape}")
print(f"  y形状: {y.shape}")
print(f"  X范围: [{X.min():.2f}, {X.max():.2f}]")
print(f"  y范围: [{y.min():.2f}, {y.max():.2f}]")
print(f"\n真实参数:")
print(f"  截距 (θ0): {TRUE_INTERCEPT}")
print(f"  斜率 (θ1): {TRUE_SLOPE}")

### 步骤3: 数据可视化

先看看生成的数据分布情况：

In [ ]:
# ============================================================
# 数据可视化
# ============================================================

plt.figure(figsize=FIGURE_SIZE)

# 绘制散点图
plt.scatter(X, y, alpha=0.6, s=50, 
            edgecolors='k', linewidth=0.5, 
            label='数据点', color='steelblue')

# 设置标题和标签
plt.xlabel('特征 X', fontsize=12)
plt.ylabel('目标 y', fontsize=12)
plt.title('训练数据分布', fontsize=14, fontweight='bold')

# 添加网格和图例
plt.grid(True, alpha=0.3, linestyle='--')
plt.legend(loc='best')

plt.tight_layout()
plt.show()

print("✓ 数据可视化完成")

---

### 步骤4: 方法1 - 使用正规方程求解

正规方程公式：$\theta = (X^TX)^{-1}X^Ty$

这是线性回归的解析解，可以直接计算出最优参数。

In [ ]:
# ============================================================
# 方法1: 正规方程求解
# ============================================================

# 步骤1: 添加偏置项 x0 = 1
# 将原始特征X扩展为[1, X]的形式，以便包含截距项
X_b = np.c_[np.ones((N_SAMPLES, 1)), X]  # shape: (100, 2)
print(f"扩展后的X_b形状: {X_b.shape}")
print(f"X_b前3行:\n{X_b[:3]}")

# 步骤2: 应用正规方程计算最优参数
# 公式: θ = (X^T * X)^(-1) * X^T * y

# 2.1 计算 X^T * X
X_transpose_X = X_b.T.dot(X_b)  # shape: (2, 2)
print(f"\nX^T * X 的形状: {X_transpose_X.shape}")

# 2.2 计算 (X^T * X)^(-1)
X_transpose_X_inv = np.linalg.inv(X_transpose_X)

# 2.3 计算最终参数 θ = (X^T * X)^(-1) * X^T * y
theta_best = X_transpose_X_inv.dot(X_b.T).dot(y)

print("\n" + "="*50)
print("方法1: 正规方程结果")
print("="*50)
print(f"估计的截距 (θ0): {theta_best[0][0]:.4f}  (真实值: {TRUE_INTERCEPT})")
print(f"估计的斜率 (θ1): {theta_best[1][0]:.4f}  (真实值: {TRUE_SLOPE})")
print(f"截距误差: {abs(theta_best[0][0] - TRUE_INTERCEPT):.4f}")
print(f"斜率误差: {abs(theta_best[1][0] - TRUE_SLOPE):.4f}")

### 步骤5: 使用估计的参数进行预测

In [ ]:
# ============================================================
# 使用正规方程结果进行预测
# ============================================================

# 创建新的测试数据点
X_new = np.array([[0], [2]])  # 两个测试点：x=0 和 x=2
X_new_b = np.c_[np.ones((2, 1)), X_new]  # 添加偏置项

# 进行预测: y_pred = X_new_b * theta
y_predict = X_new_b.dot(theta_best)

print("预测结果:")
print(f"  x=0 时，预测 y = {y_predict[0][0]:.4f}")
print(f"  x=2 时，预测 y = {y_predict[1][0]:.4f}")

### 步骤6: 可视化拟合结果

In [ ]:
# ============================================================
# 拟合结果可视化
# ============================================================

plt.figure(figsize=FIGURE_SIZE)

# 绘制拟合线
plt.plot(X_new, y_predict, 'r-', linewidth=2, label='拟合线 (正规方程)')

# 绘制数据点
plt.scatter(X, y, alpha=0.6, s=50, 
            edgecolors='k', linewidth=0.5,
            label='数据点', color='steelblue')

# 设置坐标轴范围
plt.axis([0, 2, 0, 15])

# 设置标题和标签
plt.xlabel('特征 X', fontsize=12)
plt.ylabel('目标 y', fontsize=12)
plt.title('线性回归拟合结果 (正规方程法)', fontsize=14, fontweight='bold')

# 添加参数文本
param_text = f'θ0 = {theta_best[0][0]:.3f}\nθ1 = {theta_best[1][0]:.3f}'
plt.text(0.1, 13, param_text, fontsize=11, 
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.grid(True, alpha=0.3, linestyle='--')
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()

---

### 步骤7: 方法2 - 使用Scikit-learn

Scikit-learn提供了简单易用的LinearRegression类，内部使用最小二乘法优化。

In [ ]:
# ============================================================
# 方法2: 使用Scikit-learn
# ============================================================

# 创建线性回归模型
lin_reg = LinearRegression()

# 训练模型
lin_reg.fit(X, y)

# 获取参数
print("="*50)
print("方法2: Scikit-learn结果")
print("="*50)
print(f"估计的截距 (θ0): {lin_reg.intercept_[0]:.4f}")
print(f"估计的斜率 (θ1): {lin_reg.coef_[0][0]:.4f}")

# 使用模型预测
y_pred_sklearn = lin_reg.predict(X_new)
print(f"\n预测结果:")
print(f"  x=0 时，预测 y = {y_pred_sklearn[0][0]:.4f}")
print(f"  x=2 时，预测 y = {y_pred_sklearn[1][0]:.4f}")

### 步骤8: 方法对比

比较两种方法的结果，验证它们是否一致：

In [ ]:
# ============================================================
# 方法对比
# ============================================================

print("="*60)
print("两种方法对比")
print("="*60)
print(f"{'参数':<15} {'正规方程':>15} {'Scikit-learn':>15}")
print("-"*60)
print(f"{'截距 (θ0)':<15} {theta_best[0][0]:>15.6f} {lin_reg.intercept_[0]:>15.6f}")
print(f"{'斜率 (θ1)':<15} {theta_best[1][0]:>15.6f} {lin_reg.coef_[0][0]:>15.6f}")
print("-"*60)

# 计算差异
intercept_diff = abs(theta_best[0][0] - lin_reg.intercept_[0])
slope_diff = abs(theta_best[1][0] - lin_reg.coef_[0][0])

print(f"\n参数差异:")
print(f"  截距差异: {intercept_diff:.10f}")
print(f"  斜率差异: {slope_diff:.10f}")

if intercept_diff < 1e-10 and slope_diff < 1e-10:
    print("\n✓ 两种方法的结果完全一致！")

---

### 步骤9: 补充 - 使用伪逆矩阵

NumPy提供了`np.linalg.lstsq`和`np.linalg.pinv`函数，它们使用奇异值分解(SVD)来计算伪逆，数值上更加稳定。

In [ ]:
# ============================================================
# 补充方法: 使用NumPy的lstsq和伪逆
# ============================================================

# 方法3: 使用np.linalg.lstsq (Scikit-learn内部实际使用的方法)
theta_lstsq, residuals, rank, s = np.linalg.lstsq(X_b, y, rcond=None)

print("方法3: np.linalg.lstsq")
print(f"  截距: {theta_lstsq[0][0]:.6f}")
print(f"  斜率: {theta_lstsq[1][0]:.6f}")

# 方法4: 使用伪逆 np.linalg.pinv
theta_pinv = np.linalg.pinv(X_b).dot(y)

print("\n方法4: np.linalg.pinv")
print(f"  截距: {theta_pinv[0][0]:.6f}")
print(f"  斜率: {theta_pinv[1][0]:.6f}")

print("\n✓ 所有方法得到相同的结果！")

---

## 📝 总结

### 关键要点

1. ✅ **线性回归**是建立特征和目标之间线性关系的基本算法
2. ✅ **正规方程** $\theta = (X^TX)^{-1}X^Ty$ 提供了解析解
3. ✅ 使用正规方程时，需要在特征矩阵前添加**偏置项**（一列全1）
4. ✅ **Scikit-learn**的LinearRegression内部使用最小二乘法，结果与正规方程一致
5. ✅ 对于数值稳定性，可以使用**伪逆矩阵**代替直接求逆

### 何时使用正规方程 vs 梯度下降

| 方法 | 适用场景 | 特点 |
|-----|---------|------|
| 正规方程 | 特征数量 < 10,000 | 一步求解，无需迭代 |
| 梯度下降 | 特征数量很大 | 迭代求解，需要调参 |

### 下一步学习

- 📘 多项式回归 (Polynomial Regression)
- 📙 岭回归 (Ridge Regression)
- 📕 Lasso回归
- 📗 梯度下降算法实现

### 练习题

1. 尝试生成更多噪声的数据，观察模型性能变化
2. 实现批量梯度下降算法求解线性回归
3. 使用真实数据集（如波士顿房价）进行回归

## 📚 参考资料

- [Scikit-learn文档: LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
- [NumPy文档: linalg.lstsq](https://numpy.org/doc/stable/reference/generated/numpy.linalg.lstsq.html)
- 《机器学习实战》第4章